###Transmembrane Voltage Analysis Notebook###
This notebook analyzes all sensor data files from within a specified folder and saves the results for later use.

To Use:
- Read the steps for a general idea of what happens in each cell and run each section one at a time.
- Pay attention for green comments in the code as they might require your attention.



---



###Step 1: Install Necessary Libraries and Mount Google Drive###
This cell installs all required Python libraries and mounts your Google Drive so the notebook can access your files.

If asked for perimissions, sign in to your Google account and allow access to your Drive.

You should see a message confirming the successful mounting of your Google Drive.

In [ ]:
!pip install -q pyabf matplotlib numpy pandas openpyxl
from google.colab import drive
import os
import pandas as pd
import pyabf
import matplotlib.pyplot as plt
import numpy as np
import openpyxl
from openpyxl.drawing.image import Image

drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
Mounted at /content/drive


###Step 2: Choose and verify the folder location###
This cell checks if the correct folder path is provided and that such a path is present in your Google Drive.

Change the folder path to whichever folder you would like to analyze and run the cell.

You should see a message confirming that the folder you are looking for exists.

In [ ]:
#Make sure to move the shared summer folder into 'My Drive'
#You can drag and drop the folder from the 'Shared with me' to 'My Drive'
shared_folder_path = '/content/drive/My Drive/Summer2025/ResearchData/TransmembraneVoltage/'
#Specify the data folder you want analyzed.
#Example: data_folder = 'Yellow(Mid)'
data_folder = 'Yellow(Mid)'
folder_path = os.path.join(shared_folder_path, data_folder)
if os.path.exists(folder_path):
    print(f"The folder '{folder_path}' exists.")
else:
    print(f"The folder '{folder_path}' does not exist. Make sure Lab folder is moved to My Drive.")

The folder '/content/drive/My Drive/Summer2025/ResearchData/TransmembraneVoltage/Yellow(Mid)' exists.


###Step 3: Locate Data Files###
This cell scans the specified folder for sensor data ('.abf' files) and lists how many were found.

You should see a message confirming the ammount of data files found within the folder.

In [ ]:
abf_files = [f for f in os.listdir(folder_path) if f.endswith('.abf')]
print(f"{len(abf_files)} data files found.")

145 data files found.


###Step 4: Analyze ABF Files and Generate Plots###
This sections reads each file, extracts data, performs linear regression, and generates plots for each file.

Wait for the cell to complete; it may take some time depending on the number of files in the folder.

Expect to see a dataframe summarizing slopes, intercepts, and other statistics. Also expect to see a plot for each file in the folder.

These will be saved to the shared folder in a clearer format in the next step. *When reviewing the graphs, pay attention to the scale on the y-axis.*

In [ ]:
plot_names = []
df = pd.DataFrame(columns=['File Name', 'Slope 1', 'Slope 2', 'Slope 3', 'y-int 1', 'y-int 2', 'y-int 3', 'x-int 1', 'x-int 2', 'x-int 3', 'avg x-int', 'std dev x-int'])
for abf_file in abf_files:
  try:
    abf_path = os.path.join(folder_path, abf_file)
    abf = pyabf.ABF(abf_path)
    cache = []
    plt.figure(figsize=(10, 6))
    for sweep in abf.sweepList:
      abf.setSweep(sweep)
      epochs = abf.sweepEpochs
      start = epochs.p1s[2]
      end = epochs.p1s[3]
      x = abf.sweepC[start:end]
      y = abf.sweepY[start:end]
      m, b = np.polyfit(x, y, 1)
      cache.append(m)
      cache.append(b)
      cache.append(-b/m)
      fit_line = m*x + b
      plt.plot(x, y, 'o', alpha=0.5, markersize=2)
      plt.plot(x, fit_line, color='black', alpha=0.7)
    plt.xlabel('Millivolts (mV)')
    plt.ylabel('Picoamps (pA)')
    plt.title(abf_file[:-4])
    plt.grid(True)
    plt.gcf().patch.set_edgecolor('black')
    plt.gcf().patch.set_linewidth(1)
    plt.tight_layout()
    plot_names.append(abf_file[:-4]+'.png')
    plt.savefig(abf_file[:-4]+'.png')
    plt.close()
    val = [abf_file, cache[0], cache[3], cache[6], cache[1], cache[4], cache[7], cache[2], cache[5], cache[8], np.mean(cache[2:12:3]), np.std(cache[2:12:3])]
    df.loc[len(df)] = val
  except:
    print(f"Error processing {abf_file}")
    df.loc[len(df)] = [abf_file, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']
#print(df)

###Step 5: Save Results to an Excel File###
This cell saves numerical and graphical results into an Excel file with two sheets.

Expect to see a spreadsheet in the same lab folder as the data with the saved results.

*NOTE: The available results spreadsheet will reflect the latest analysis of that specific folder. When new data is added into a folder, you should run this program again to replace the old results.*

In [ ]:
file_name = folder_path+'/Colab_Notebook_Results.xlsx'
df.to_excel(file_name, index=False)

wb = openpyxl.load_workbook(file_name)
sheet = wb['Sheet1']
sheet.title = 'Numerical Results'
new_sheet = wb.create_sheet(title='Graphical Results')
wb.active = wb.index(new_sheet)
ws = wb.active
loc = 1
for plot_name in plot_names:
  img = Image(plot_name)
  new_sheet[f"A{loc}"] = plot_name[:-4]
  new_sheet.add_image(img, f"A{loc+1}")
  loc += 31
wb.save(file_name)